### Cross Validation Penalty Parameter
Einfaches Skript für die Cross Validation der Penalty Parameter auf unseren Daten.

#### Benötigte Matrizen
* ```data/matrices/matrix_cla_incl_current.pickle```
* ```data/matrices/matrix_cla_incl_history.pickle```
* ```data/matrices/matrix_reg_incl_current.pickle```
* ```data/matrices/matrix_reg_incl_history.pickle```

In [13]:
from sklearn import svm
from sklearn.grid_search import GridSearchCV
from imports.matrix_helper import MatrixHelper
from prettytable import PrettyTable

matrix_helper = MatrixHelper()

table = PrettyTable(['Classifier', 'With FH', 'Penalty Parameter'])
table.align["Classifier"] = "l"

param_grid = {'C': [0.01, 0.1, 0.5, 1, 2]}

models = [('cla', 'SVC', 'current'), ('cla', 'SVC', 'history'), ('reg', 'SVR', 'current'), ('reg', 'SVR', 'history')]
for model in models:
    matrices = matrix_helper.load_from_parse('data/matrices/matrix_{}_incl_{}.pickle'.format(model[0], model[2]))
    feature_matrix = matrices[0]
    features_count = feature_matrix.shape[1] - 1
    
    if model[1] == 'SVC':
        classifier = svm.LinearSVC()
    else:
        classifier = svm.LinearSVR()

    grid_search = GridSearchCV(classifier, param_grid, cv=5)
    grid_search.fit(feature_matrix[:, range(features_count)], feature_matrix[:, features_count])

    table.add_row([model[1], 'Yes' if model[2] == 'history' else 'No', grid_search.best_params_])

print(table)

+------------+---------+-------------------+
| Classifier | With FH | Penalty Parameter |
+------------+---------+-------------------+
| SVC        |    No   |     {'C': 0.1}    |
| SVC        |   Yes   |     {'C': 0.1}    |
| SVR        |    No   |     {'C': 0.5}    |
| SVR        |   Yes   |     {'C': 0.1}    |
+------------+---------+-------------------+
